In [21]:
import pandas as pd

# user-defined modules
import sys
module_path = "D:\PythonProject\data-gatherer\common"
sys.path.append(module_path)
import dbconnect

In [22]:
beg_date = '20120101'
end_date = '20221231'

engine = dbconnect.db_engine("DEMO_DW")
query = """SELECT date_id, ta_avg, ta_max, ta_min, ta_diff \
             FROM weather WHERE date_id between %(date1)s and %(date2)s ORDER BY date_id;"""

# sql execution with parameters
df_weather = pd.read_sql(sql=query, con=engine, params={'date1' : beg_date, 'date2' : end_date})

In [23]:
df_weather

,date_id,ta_avg,ta_max,ta_min,ta_diff
0,20120101,1.2,5.6,-3.1,8.7
1,20120102,-1.0,4.5,-4.4,8.9
2,20120103,-0.1,3.8,-2.6,6.4
3,20120104,-3.2,-0.4,-4.8,4.4
4,20120105,-2.1,3.1,-5.6,8.7
...,...,...,...,...,...
4013,20221227,0.0,6.6,-7.2,13.8
4014,20221228,1.0,5.1,-2.0,7.1
4015,20221229,-0.3,4.3,-3.3,7.6
4016,20221230,0.2,4.4,-3.9,8.3


In [24]:
df_weather.shape

(4018, 5)

In [26]:
#이상치 제거
index0 = df_weather[df_weather['ta_avg'] <= -50].index 
# print(index0) 
df_weather = df_weather.drop(index0)

In [27]:
df_weather.shape

(4016, 5)

In [28]:
df_weather.insert(1, 'year', df_weather['date_id'].str[0:4])
df_weather.insert(2, 'month', df_weather['date_id'].str[4:6])
# df_fx_all_save = df_fx_all_save.drop('cur_unit', axis=1)

In [29]:
df_weather

,date_id,year,month,ta_avg,ta_max,ta_min,ta_diff
0,20120101,2012,01,1.2,5.6,-3.1,8.7
1,20120102,2012,01,-1.0,4.5,-4.4,8.9
2,20120103,2012,01,-0.1,3.8,-2.6,6.4
3,20120104,2012,01,-3.2,-0.4,-4.8,4.4
4,20120105,2012,01,-2.1,3.1,-5.6,8.7
...,...,...,...,...,...,...,...
4013,20221227,2022,12,0.0,6.6,-7.2,13.8
4014,20221228,2022,12,1.0,5.1,-2.0,7.1
4015,20221229,2022,12,-0.3,4.3,-3.3,7.6
4016,20221230,2022,12,0.2,4.4,-3.9,8.3


In [42]:
# pip install plotly
import plotly.express as px
import webbrowser

만일 plotly 패키지의 코드가 잘 동작하지 않을 경우에는 command line에서 다음 코드 실행
> pip install --upgrade nbformat

In [39]:
beg_year = beg_date[0:4]
end_year = end_date[0:4]

In [43]:
# points: all or outliers
month = '07'
graph_title = f'Monthly Weather <Month: {month}> (from {beg_year} to {end_year})'
df_weather_month = df_weather.query('month == @month')
fig = px.box(df_weather_month, title=graph_title, x='year', y='ta_avg', points='all', hover_data=df_weather_month.columns, color='month')
fig.show()
file_path = "D:\PythonProject\data-gatherer\kma\output\날씨_월별_비교.html"
fig.write_html(file_path)
webbrowser.open_new_tab(file_path)

True

### 연월(year-month) 기온 평균

In [14]:
df_weather_ym = df_weather.groupby(['year', 'month']).agg(mean_ta_avg = ('ta_avg', 'mean'),
                                                          mean_ta_max = ('ta_max', 'mean'),
                                                          mean_ta_min = ('ta_min', 'mean'),
                                                          mean_ta_diff = ('ta_diff', 'mean'))

In [15]:
df_weather_ym

mean_ta_avg  mean_ta_max  mean_ta_min  mean_ta_diff
year month                                                     
2012 01        0.841935     5.112903    -2.819355      7.932258
     02        1.075862     5.937931    -3.175862      9.113793
     03        7.674194    12.712903     3.493548      9.219355
     04       15.080000    21.316667     9.533333     11.783333
     05       20.116129    26.154839    15.070968     11.083871
...                 ...          ...          ...           ...
2022 08       26.925806    31.354839    23.364516      7.990323
     09       21.886667    26.836667    17.746667      9.090000
     10       15.548387    21.606452    10.141935     11.464516
     11       11.046667    17.713333     5.226667     12.486667
     12        0.396774     5.390323    -4.029032      9.419355

[132 rows x 4 columns]

In [16]:
df_weather_ym.reset_index(inplace=True)

In [17]:
df_weather_ym

,year,month,mean_ta_avg,mean_ta_max,mean_ta_min,mean_ta_diff
0,2012,01,0.841935,5.112903,-2.819355,7.932258
1,2012,02,1.075862,5.937931,-3.175862,9.113793
2,2012,03,7.674194,12.712903,3.493548,9.219355
3,2012,04,15.080000,21.316667,9.533333,11.783333
4,2012,05,20.116129,26.154839,15.070968,11.083871
...,...,...,...,...,...,...
127,2022,08,26.925806,31.354839,23.364516,7.990323
128,2022,09,21.886667,26.836667,17.746667,9.090000
129,2022,10,15.548387,21.606452,10.141935,11.464516
130,2022,11,11.046667,17.713333,5.226667,12.486667


In [19]:
# points: all or outliers
fig = px.line(df_weather_ym, x='year', y='mean_ta_avg', hover_data=df_weather_ym.columns, color = 'month')
fig.show() #파일을 시각화 하는 코드
# fig.write_html("D:\PythonProject\data-gatherer\kma\output\날씨_월별_비교.html") #파일을 저장하는 코드